<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL-DP-PSO-SGD/FL_DP_PSO_SGD_MNIST_velocity_clipping_Clients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pega o erro de cada um e compara. A partícula com menor erro pede o gbest.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import csv
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math

In [2]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=128, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [3]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 20
NUM_RODADAS = 10
INERCIA, C1, C2 = 0.9, 0.8, 0.9
EPSILON = 6/math.sqrt(10)
DELTA = 1e-5
SENSITIVITY = 3
MAX_VELOCITY = 1.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE, hidden_size=256)
criterio = nn.CrossEntropyLoss()

training on cuda


In [4]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)


# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.pesos = copy.deepcopy(modelo_cliente.state_dict())  # Conjunto de pesos da partícula
        self.melhor_pesos = copy.deepcopy(self.pesos)  # pbest (melhor posição da partícula)
        self.melhor_erro = float('inf')  # Melhor erro alcançado
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}  # Velocidade do PSO
        self.device = modelo_cliente.device  # Dispositivo do modelo

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY):
        """Atualiza os pesos da partícula usando a equação do PSO."""
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name] +
                C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name]) +
                C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )
            self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)
            sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON

            # Gerar ruído diretamente com a distribuição normal do PyTorch (muito mais eficiente!)
            noise = torch.normal(mean=0, std=sigma, size=self.velocidade[name].shape, device=self.device)
            self.velocidade[name] += noise
            #clipping velocity
            self.pesos[name] += self.velocidade[name]

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        """Calcula a perda da partícula no modelo do cliente."""
        modelo_cliente.load_state_dict(self.pesos)  # Aplica os pesos da partícula no modelo do cliente
        modelo_cliente.eval()
        total_loss = 0
        loader = DataLoader(dados, batch_size=64, shuffle=False)

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(modelo_cliente.device), labels.to(modelo_cliente.device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(loader)


class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, num_particulas=5):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:

            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()

    def refinar_com_adam(self, criterio):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        otimizador = optim.Adam(self.modelo.parameters(), lr=0.01, weight_decay=1e-5)
        train_loader = DataLoader(self.dados, batch_size=32, shuffle=True)

        self.modelo.train()
        for _ in range(5):  # 10 épocas de refinamento com Adam
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.modelo.device), labels.to(self.modelo.device)
                otimizador.zero_grad()
                outputs = self.modelo(inputs)
                loss = criterio(outputs, labels)
                loss.backward()
                otimizador.step()

        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        self.melhor_particula.pesos = copy.deepcopy(self.modelo.state_dict())

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = copy.deepcopy(modelo_global.state_dict())  # Inicializa com o modelo global
    melhor_erro_global = float('inf')

    for rodada in range(num_rodadas):
        resultados_rodada = []

        for cliente in clientes:
            cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY)  # Treino com PSO
            cliente.refinar_com_adam(criterio)  # Refinamento com Adam
            erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
            resultados_rodada.append((cliente.cliente_id, erro_cliente))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        melhor_cliente = random.choice(top_3_results)
        melhor_cliente_id = melhor_cliente[0]
        melhor_erro_cliente = melhor_cliente[1]

        melhor_peso_global = copy.deepcopy(clientes[melhor_cliente_id].melhor_particula.pesos)
        melhor_erro_global = melhor_erro_cliente

        modelo_global.load_state_dict(melhor_peso_global)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)


        print(f"Rodada {rodada+1}/{num_rodadas}: Cliente {melhor_cliente_id} enviou os pesos.")
        print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print(f"Teste -> Perda: {test_loss:.4f}, Acurácia: {test_accuracy:.2f}%\n")

    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação

    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples) * 100

    return test_loss, test_accuracy


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_train = mnist_train.targets.numpy()  # Labels

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_test = mnist_test.targets.numpy()  # Labels

# Dividir treino e teste manualmente como no Iris

# Normalizar como no Iris
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets como no Iris
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)
SUBSET_SIZE = 12000

trainloaders = [create_subset(trainset, SUBSET_SIZE) for _ in range(NUM_CLIENTES)]

testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Criando os clientes
clientes = [Cliente(i, modelo_global, trainloaders[i], NUM_PARTICULAS) for i in range(NUM_CLIENTES)]

# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader, EPSILON, DELTA, SENSITIVITY, MAX_VELOCITY)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 488kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.83MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Rodada 1/10: Cliente 2 enviou os pesos.
Erro Global Atualizado: 1564509.1569
Teste -> Perda: 86995.5569, Acurácia: 87.51%

Rodada 2/10: Cliente 0 enviou os pesos.
Erro Global Atualizado: 1472380.1895
Teste -> Perda: 82967.2253, Acurácia: 87.04%

Rodada 3/10: Cliente 1 enviou os pesos.
Erro Global Atualizado: 1560759.3251
Teste -> Perda: 95143.2394, Acurácia: 87.34%

Rodada 4/10: Cliente 0 enviou os pesos.
Erro Global Atualizado: 1472380.1895
Teste -> Perda: 86215.7275, Acurácia: 87.18%

Rodada 5/10: Cliente 0 enviou os pesos.
Erro Global Atualizado: 1472380.1895
Teste -> Perda: 82591.1133, Acurácia: 87.09%

Rodada 6/10: Cliente 0 enviou os pesos.
Erro Global Atualizado: 1472380.1895
Teste -> Perda: 84563.5583, Acurácia: 87.04%

Rodada 7/10: Cliente 1 enviou os pesos.
Erro Global Atualizado: 1560759.3251
Teste -> Perda: 96362.7364, Acurácia: 87.17%

Rodada 8/10: Cliente 2 enviou os pesos.
Erro Global Atualizado: 